In [2]:
import nrrd
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
import random

In [3]:
testlist = [18]

In [4]:
for number in testlist:

    ca, _ = nrrd.read("../../CA/"+str(number)+".ca.seg.nrrd")
    dilation_size = 3
    element = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2 * dilation_size + 1, 2 * dilation_size + 1), (dilation_size, dilation_size))
    for i in range(0, ca.shape[2]):
        ca[:,:,i] = cv2.dilate(ca[:,:,i], element)
    ca_copy = ca.copy()
    full_ct, _ =  nrrd.read("../../ESUS/"+str(number)+".img.nrrd")
    masked = ca * full_ct
    masked = (masked-np.min(masked)) / (np.max(masked)-np.min(masked))
    gt, _ =  nrrd.read("../../ESUS/"+str(number)+".b.seg.nrrd")
    gt[gt>1] = 1 # make all plaque segments 1 instead of a new number for each one

    patch_coords = [gt.shape]

    current_volume = 1
    for i in range(0, ca.shape[2]):
        if np.sum(ca[:,:,i]) > 0:
            
            contours, _ = cv2.findContours(ca[:,:,i], mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_SIMPLE)
            for c in contours:
                try:
                    # compute the center of the contour
                    M = cv2.moments(c)
                    cX = int(M["m10"] / M["m00"])
                    cY = int(M["m01"] / M["m00"])
                    
                    volume = masked[int(cY-32):int(cY+32), int(cX-32):int(cX+32), i:i+64]
                    nrrd.write("./test_data/reform/inputs/"+str(number)+"_volume_"+str(current_volume)+".nrrd", volume)

                    volume_gt = gt[int(cY-32):int(cY+32), int(cX-32):int(cX+32), i:i+64]
                    nrrd.write("./test_data/reform/gt/"+str(number)+"_volume_"+str(current_volume)+".nrrd", volume_gt)

                    ca[int(cY-32):int(cY+32), int(cX-32):int(cX+32), i:i+64] = 0

                    patch_coords.append([current_volume, int(cY-32), int(cY+32), int(cX-32), int(cX+32), i, i+64])
                    #print(int(cY-32), int(cY+32), int(cX-32), int(cX+32), i, i+64)
                    
                    current_volume += 1
                except:
                    pass

In [5]:
patch_coords

[(512, 512, 211),
 [1, 311, 375, 179, 243, 27, 91],
 [2, 243, 307, 139, 203, 33, 97],
 [3, 209, 273, 143, 207, 40, 104],
 [4, 188, 252, 175, 239, 54, 118],
 [5, 220, 284, 175, 239, 66, 130],
 [6, 168, 232, 137, 201, 88, 152],
 [7, 311, 375, 145, 209, 90, 154],
 [8, 152, 216, 172, 236, 103, 167],
 [9, 186, 250, 175, 239, 118, 182],
 [10, 124, 188, 204, 268, 121, 185],
 [11, 328, 392, 177, 241, 122, 186],
 [12, 276, 340, 162, 226, 122, 186],
 [13, 173, 237, 207, 271, 129, 193]]

In [6]:
import json

with open("patch_coords.json", 'w') as f:
    # indent=2 is not needed but makes the file human-readable 
    # if the data is nested
    json.dump(patch_coords, f, indent=2) 

with open("patch_coords.json", 'r') as f:
    p_coords = json.load(f)

print(p_coords)

[[512, 512, 211], [1, 311, 375, 179, 243, 27, 91], [2, 243, 307, 139, 203, 33, 97], [3, 209, 273, 143, 207, 40, 104], [4, 188, 252, 175, 239, 54, 118], [5, 220, 284, 175, 239, 66, 130], [6, 168, 232, 137, 201, 88, 152], [7, 311, 375, 145, 209, 90, 154], [8, 152, 216, 172, 236, 103, 167], [9, 186, 250, 175, 239, 118, 182], [10, 124, 188, 204, 268, 121, 185], [11, 328, 392, 177, 241, 122, 186], [12, 276, 340, 162, 226, 122, 186], [13, 173, 237, 207, 271, 129, 193]]


In [9]:
# Start Reforming

In [9]:
reformation = np.zeros(shape=(p_coords[0]))

In [15]:
number = 18
for patch in range(1, len(p_coords)):
    print(p_coords[patch])
    patch_path = "./test_data/reform/gt/"+str(number)+"_volume_"+str(p_coords[patch][0])+".nrrd"
    try:
        patch_data, _ = nrrd.read(patch_path)
        reformation[p_coords[patch][1]:p_coords[patch][2], p_coords[patch][3]:p_coords[patch][4], p_coords[patch][5]:p_coords[patch][6]] += patch_data
    except:
        pass
reformation[reformation > 1] = 1

[1, 311, 375, 179, 243, 27, 91]
[2, 243, 307, 139, 203, 33, 97]
[3, 209, 273, 143, 207, 40, 104]
[4, 188, 252, 175, 239, 54, 118]
[5, 220, 284, 175, 239, 66, 130]
[6, 168, 232, 137, 201, 88, 152]
[7, 311, 375, 145, 209, 90, 154]
[8, 152, 216, 172, 236, 103, 167]
[9, 186, 250, 175, 239, 118, 182]
[10, 124, 188, 204, 268, 121, 185]
[11, 328, 392, 177, 241, 122, 186]
[12, 276, 340, 162, 226, 122, 186]
[13, 173, 237, 207, 271, 129, 193]


In [16]:
nrrd.write("./test_data/reform/predictions/test_gt_reform.nrrd", reformation)